# 1. 确认GPU

In [1]:
!nvidia-smi

Thu Jun 19 22:10:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...    Off |   00000000:01:00.0  On |                  N/A |
| N/A   37C    P5             11W /   55W |     912MiB /   8188MiB |     36%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

(device(type='cpu'),
 <torch.cuda.device at 0x7401f8317fa0>)

In [4]:
print(torch.cuda.device_count())

1


In [5]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回gpu"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]。"""
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]      
    return devices if devices else [torch.device('cpu')]
                    
print(try_gpu())
print(try_gpu(10))
print(try_all_gpus())

cuda:0
cpu
[device(type='cuda', index=0)]


In [6]:
# 查询张量所在设备
X = torch.tensor([1,2,3])
print(X.device) # 默认在CPU内存上

cpu


In [6]:
# 存储在GPU上
X = torch.ones(2,3,device=try_gpu())
X 

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [7]:
# 在第二个GPU上创建一个随即张量
Y = torch.rand(2,3,device=try_gpu(1))
print(Y.device) # 没有1号GPU，则放到CPU上
Y

cpu


tensor([[0.7419, 0.5102, 0.5761],
        [0.3662, 0.1303, 0.1366]])

In [12]:
# 要计算X+Y，我们需要决定在哪里执行这个操作
Z = X.cuda(0) # X+Y必须X和Y都在同一个GPU上
print(X)
print(Z)

tensor([1, 2, 3])
tensor([1, 2, 3], device='cuda:0')


In [14]:
Y = torch.rand(2,3,device=try_gpu(0))
Y + Z
Z.cuda(0) is Z # 如果变量在0号GPU时，就返回True

True

In [16]:
# 神经网络与GPU
net = nn.Sequential(nn.Linear(3,1)) # 创建神经网络时已经把权重初始化好了
net = net.to(device=try_gpu()) # 把所有参数在0号GPU上拷贝一份
X = torch.ones(2,3,device=try_gpu()) # X 在0号GPU上
net(X) # 所以前项运算所有元素都在0号GPU上运行

tensor([[-0.8234],
        [-0.8234]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [17]:
# 确认模型参数存储在同一个GPU上
net[0].weight.data.device

device(type='cuda', index=0)